# Association Rule for Store Dataset

## Load Data

Load data dan tunjukan 5 row pertamanya

Dataset :
https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv

In [2]:
!pip install mlxtend --upgrade

import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.1 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv")
df.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,tropical fruit,2015,7,21,1
1,2552,2015-05-01,whole milk,2015,5,1,4
2,2300,2015-09-19,pip fruit,2015,9,19,5
3,1187,2015-12-12,other vegetables,2015,12,12,5
4,3037,2015-01-02,whole milk,2015,1,2,4


<h2> EDA

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
 3   year             38765 non-null  int64 
 4   month            38765 non-null  int64 
 5   day              38765 non-null  int64 
 6   day_of_week      38765 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.1+ MB


In [13]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
38760    False
38761    False
38762    False
38763    False
38764    False
Length: 38765, dtype: bool

In [14]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38006 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38006 non-null  int64 
 1   Date             38006 non-null  object
 2   itemDescription  38006 non-null  object
 3   year             38006 non-null  int64 
 4   month            38006 non-null  int64 
 5   day              38006 non-null  int64 
 6   day_of_week      38006 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.3+ MB


<h2> Data Preprocessing

Ubahlah dataset di atas menjadi bentuk one hot encoding. Asumsikanlah bahwa pembelian barang yang dilakukan oleh Customer ID yang sama dalam hari yang sama terjadi dalam 1 transaksi yang sama (satu Order ID)

contoh :
![image-2.png](attachment:image-2.png)

In [15]:
df.rename(columns={'Member_number': 'customer_id', 'Date': 'date', 'itemDescription': 'item_name', }, inplace=True)
df['order_id'] = df['customer_id'].astype(str) +"_"+ df["date"].astype(str)
df.head()

,customer_id,date,item_name,year,month,day,day_of_week,order_id
0,1808,2015-07-21,tropical fruit,2015,7,21,1,1808_2015-07-21
1,2552,2015-05-01,whole milk,2015,5,1,4,2552_2015-05-01
2,2300,2015-09-19,pip fruit,2015,9,19,5,2300_2015-09-19
3,1187,2015-12-12,other vegetables,2015,12,12,5,1187_2015-12-12
4,3037,2015-01-02,whole milk,2015,1,2,4,3037_2015-01-02


In [16]:
def getProductList(row):
  return [product for product in row.unique()]

data = df[['order_id', 'item_name']].groupby('order_id').agg(getProductList).reset_index()
cols = data['order_id']
vals = data['item_name']

In [17]:
te = TransactionEncoder()
arr = te.fit(vals).transform(vals)
df = pd.DataFrame(arr, columns = te.columns_)
df.insert(0, 'order_id', cols)
df.head()

,order_id,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1000_2014-06-24,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,1000_2015-03-15,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,1000_2015-05-27,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1000_2015-07-24,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1000_2015-11-25,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Apriori Algorithm

Gunakanlah algroitma apriori untuk mencari tahu produk mana yang sering dibeli secara bersama-sama (min support = 0.01)

In [18]:
from mlxtend.frequent_patterns import apriori, association_rules
df.drop('order_id', axis = 1, inplace = True)
frq_items = apriori(df, min_support = 0.01, use_colnames = True)
frq_items

,support,itemsets
0,0.021386,(UHT-milk)
1,0.033950,(beef)
2,0.021787,(berries)
3,0.016574,(beverages)
4,0.045312,(bottled beer)
...,...,...
64,0.010559,"(other vegetables, rolls/buns)"
65,0.014837,"(other vegetables, whole milk)"
66,0.013968,"(whole milk, rolls/buns)"
67,0.011629,"(soda, whole milk)"


Berdasarkan hasil di atas, buatlah association rules nya dengan syarat min confidence = 0.1 

In [19]:
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(other vegetables),(whole milk),0.122101,0.157923,0.014837,0.121511,0.769430,-0.004446,0.958551
1,(rolls/buns),(whole milk),0.110005,0.157923,0.013968,0.126974,0.804028,-0.003404,0.964550
2,(soda),(whole milk),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636
3,(yogurt),(whole milk),0.085879,0.157923,0.011161,0.129961,0.822940,-0.002401,0.967861
